In [1]:
import datetime as dt
import pandas as pd

In [2]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet")

In [3]:
df.sample(10)

,origin,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
3469,False,54400.0,1621214,1.0,[DCS00727189] GS727 FILTRO COMBUSTIBLE - HINO ...,PVE54188 Mostrador Medellín T1/30013,2025-09-24 16:59:04,PVE54188,439578,SUCURSAL MEDELLIN,NATALI ESPINAL TRUJILLO,SUCURSALES / MEDELLÍN / Ventas Mostrador Medellín,45294,[CO9012159392] TRANSPORTES Y SUMINISTROS MONTO...
8759,BD6/OUT/49352,50300.0,1615043,1.0,[BCS00281125] GS281 FILTRO COMBUSTIBLE 1° MACK...,FBAQ14333,2025-09-16 16:55:46,FBAQ14333,438008,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas Credito Bar...,19221,[CO9004352284] INGENIEROS VIAS Y CONSTRUCCIONE...
31783,BD7/OUT/16275,52800.0,1587534,1.0,[BAC00087125] DAC087 FILTRO AIRE CABINA FREIGH...,FBUC6400,2025-08-06 22:24:08,FBUC6400,430827,SUCURSAL BUCARAMANGA,EDISSON MIGUEL CIFUENTES OSORIO,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR ...,26988,[CC37396901] MAYRA ALEJANDRA GUERRERO ORTEGA
93976,BD1\OUT\58220,560000.0,1513899,16.0,"[DAB04702025] DA4702 FILTRO AIRE 2_ CHEV FVR, ...",FCOT24029,2025-04-16 12:05:00,FCOT24029,411241,PRINCIPAL COTA,JESUS ADONAY GONZALEZ FLOREZ,EXTERNOS,3303,[CC79381173] ORLANDO RATIVA ALBERTO
18417,False,103800.0,1603795,1.0,[BCS00557125] GS557 FILTRO COMBUSTIBLE SEPARAD...,PV6E18975 Mostrador Barranquilla T3/3038,2025-09-01 14:31:53,PV6E18975,435061,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas mostrador B...,38155,[CC13499607] ALVARO GARCIA LOZANO
193952,False,22400.0,1394265,1.0,[DCS00235189] GS235 FILTRO COMBUSTIBLE - TOYOT...,PV2E33974 Mostrador Calle 6 T3/20502,2024-10-11 15:33:20,PV2E33974,379920,SUCURSAL CALLE 6,ANDRES FELIPE ORDOÑEZ GREGORY,SUCURSALES / Ventas Mostrador Cll6,19957,[CC80391158] GILDARDO RIVEROS BAQUERO
189482,BD5\OUT\55871,142800.0,1399196,2.0,[DAP02020025] DA2020 FILTRO AIRE DONSSON - RET...,FMDE731,2024-10-22 12:37:30,FMDE731,381350,SUCURSAL MEDELLIN,NATALI ESPINAL TRUJILLO,SUCURSALES / MEDELLÍN / Ventas Credito Medellín,14332,[CO9011057015] ASERPARTES S.A.S.
55681,False,23000.0,1558268,1.0,[BLS00169125] GS169 FILTRO ACEITE FILTROF o TR...,PV4E15381 Mostrador Norte T2/12495,2025-06-26 14:54:51,PV4E15381,423231,SUCURSAL NORTE,WILLIAM OVALLE,SUCURSALES / Ventas Mostrador Norte,11185,[CO9004305745] CONDUFLUIDOS SAS
174786,BD1\OUT\56214,84800.0,1416998,2.0,[DAB09056025] DA9056 FILTRO AIRE 2o. JOHN DEER...,FCOT21740,2024-11-19 14:36:25,FCOT21740,385974,PRINCIPAL COTA,ANDRES CORRALES,EXTERNOS,27059,[CC93085458] MANUEL GUILLERMO RUBIO ARBELAEZ
171419,False,131300.0,1420753,1.0,[DAB12524025] DA2524UHE FILTRO AIRE ALTA EFICI...,PV2E35198 Mostrador Calle 6 T1/22055,2024-11-25 18:55:05,PV2E35198,387002,SUCURSAL CALLE 6,MAICOL MORALES RAMIREZ,SUCURSALES / Ventas Mostrador Cll6,5514,[CO10049084] PEDRO AGUSTIN BOTIA MEDINA


In [4]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

In [5]:
df = df[~(df["product_ref"].isna())]

In [6]:
df.head()

,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,...,partner_id_num,client_name_inv,Sucursal_tipo,client_name,origen,Bodega,product_ref,mes,año,dia
0,56400.0,1625201,3.0,[BLS10050125] GS050 FILTRO ACEITE BALDWIN - FO...,FCAL5717,2025-10-01 14:07:04,FCAL5717,440706,SUCURSAL CALI,CLAUDIA ORTIZ,...,35670,[CO9003546375] HIDALGO E HIDALGO COLOMBIA SAS,VENDEDOR EXTERNO,HIDALGO E HIDALGO COLOMBIA SAS,BD9,SUCURSAL CALI,BLS10050125,10,2025,1
1,46800.0,1625202,3.0,[DCE00937189] G937 FILTRO COMBUSTIBLE TOYOTA H...,FCAL5717,2025-10-01 14:07:04,FCAL5717,440706,SUCURSAL CALI,CLAUDIA ORTIZ,...,35670,[CO9003546375] HIDALGO E HIDALGO COLOMBIA SAS,VENDEDOR EXTERNO,HIDALGO E HIDALGO COLOMBIA SAS,BD9,SUCURSAL CALI,DCE00937189,10,2025,1
2,56100.0,1625203,3.0,[DAE01127189] GA1127 FILTRO AIRE TOYOTA HILUX ...,FCAL5717,2025-10-01 14:07:04,FCAL5717,440706,SUCURSAL CALI,CLAUDIA ORTIZ,...,35670,[CO9003546375] HIDALGO E HIDALGO COLOMBIA SAS,VENDEDOR EXTERNO,HIDALGO E HIDALGO COLOMBIA SAS,BD9,SUCURSAL CALI,DAE01127189,10,2025,1
3,26500.0,1625200,1.0,"[BLS00189125] GS189 FILTRO ACEITE HYUNDAI, SAN...",PV3E20025 Mostrador Valladolid T2/15371,2025-10-01 14:05:04,PV3E20025,440705,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,...,3912,[CO9006470826] GL TRANSPORTE SUMINISTROS Y AGR...,MOSTRADOR,GL TRANSPORTE SUMINISTROS Y AGREGADOS S.A.S.,<NA>,SUCURSAL VALLADOLID,BLS00189125,10,2025,1
4,45200.0,1625199,1.0,[BCS00252125] GS252 FILTRO COMBUSTIBLE BOBCAT ...,PV3E20025 Mostrador Valladolid T2/15371,2025-10-01 14:05:04,PV3E20025,440705,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,...,3912,[CO9006470826] GL TRANSPORTE SUMINISTROS Y AGR...,MOSTRADOR,GL TRANSPORTE SUMINISTROS Y AGREGADOS S.A.S.,<NA>,SUCURSAL VALLADOLID,BCS00252125,10,2025,1


In [7]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")

#Tiempo promedio 1.5 minutos

Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20251001.xlsx


## CLIENTES DATA

In [17]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

client_data = models.execute_kw(
    db, uid, password,
    "res.partner", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["phone","name" ]}
)


df_cl = pd.DataFrame(client_data)




#Tarda un minuto

In [22]:
#df_cl = df_cl.drop(columns=["id"])

df_cl = df_cl.rename(columns={"name":"client_name"})

In [11]:
fecha_ini = "2025-01-01"
fecha_fin = "2025-06-30"

df["date_invoice"] = pd.to_datetime(df["date_invoice"])


df_final = df[(df["date_invoice"] >= fecha_ini) & (df["date_invoice"] <= fecha_fin)]


In [12]:
df_final["date_invoice"].min()

Timestamp('2025-01-02 12:33:39')

In [13]:
df_final["date_invoice"].max()

Timestamp('2025-06-28 18:02:21')

In [27]:
df_agru = df_final.groupby(["client_name","salesperson_name"]).agg({"price_subtotal":"sum",
                                               "invoice_name":"count"}).reset_index()

In [28]:
clientes_ene_jun  = df_agru[(df_agru["invoice_name"]>=4) & (df_agru["price_subtotal"]> 1400000 )]

clientes_ene_jun = clientes_ene_jun.rename(columns={"invoice_name":"cantidad de facturas"})



In [29]:
clientes_merge = clientes_ene_jun.merge(df_cl, on="client_name", how="left")

In [30]:
clientes_merge.to_excel("ventasmayoresalminimodesdeenerohastajunio2025.xlsx")

In [31]:
clientes_merge.head()

,client_name,salesperson_name,price_subtotal,cantidad de facturas,phone
0,100% GENUINE OIL SAS,HUGO ALEJANDRO ZUBIETA FERNANDEZ,10156800.0,27,3208681334
1,4 US COLOMBIA SAS,WILSON PELAEZ,12640200.0,101,3176408056
2,4G4 INGENIERIA S.A.S.,JAIME ANDRES CABALLERO CABALLERO,2461000.0,16,6047045072
3,750 PARTES & SERVICIOS S.A.S,NATALI ESPINAL TRUJILLO,1447700.0,16,3004962897
4,A & B AGREGADOS SAS,NEYDER CRESPO YEPES,1789700.0,23,3216985589


## VISUM

In [10]:
import pandas as pd
ruta="/home/donsson/proyectos/API/ventashistoricas105semanas.parquet"

df_h = pd.read_parquet(ruta)

In [11]:
df_h.head()

,origin,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
0,BD6/OUT/50010,"[17134, '[DAR02708025] DA2708 FILTRO AIRE DONS...",66600.0,1632926,1.0,[DAR02708025] DA2708 FILTRO AIRE DONSSON - MER...,FBAQ14514,2025-10-10 18:49:15,FBAQ14514,442640,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas Credito Bar...,30862,[CO9015258577] TRUCKS FUENTES SAS
1,BD6/OUT/50010,"[17512, '[DAE04189025] DA4189 FILTRO AIRE 2_ M...",41200.0,1632927,1.0,[DAE04189025] DA4189 FILTRO AIRE 2_ MERCEDES B...,FBAQ14514,2025-10-10 18:49:15,FBAQ14514,442640,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas Credito Bar...,30862,[CO9015258577] TRUCKS FUENTES SAS
2,BD6/OUT/50010,"[19005, '[BCS10460125] GS460 FILTRO COMBUSTIBL...",43600.0,1632923,1.0,[BCS10460125] GS460 FILTRO COMBUSTIBLE SEPARAD...,FBAQ14514,2025-10-10 18:49:15,FBAQ14514,442640,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas Credito Bar...,30862,[CO9015258577] TRUCKS FUENTES SAS
3,BD6/OUT/50010,"[23538, '[BLS00670125] GS670 FILTRO ACEITE BOB...",28100.0,1632924,1.0,"[BLS00670125] GS670 FILTRO ACEITE BOBCAT, KUBO...",FBAQ14514,2025-10-10 18:49:15,FBAQ14514,442640,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas Credito Bar...,30862,[CO9015258577] TRUCKS FUENTES SAS
4,BD6/OUT/50010,"[29494, '[DAB08144025] DA8144 FILTRO AIRE DYNA...",93800.0,1632925,1.0,"[DAB08144025] DA8144 FILTRO AIRE DYNAPAC,SANDV...",FBAQ14514,2025-10-10 18:49:15,FBAQ14514,442640,SUCURSAL BARRANQUILLA,NEYDER CRESPO YEPES,SUCURSALES / BARRANQUILLA / Ventas Credito Bar...,30862,[CO9015258577] TRUCKS FUENTES SAS


In [12]:
df_h["cl_name"] = df_h["client_name_inv"].str.replace(r"^\[[^\]]+\]\s*", "", regex=True)

In [13]:
clientes = [
    "VILMAR LUBRICANTES S.A.S.",
    "VISUM LTDA",
    "ESTACION Y LUBRICANTES CASABLANCA S.A.S",
    "ESTACION Y LUBRICANTES SAN ANTONIO GNV S.A.S.",
    "SODATRANS LTDA"
]

df_clientes = df_h[df_h["cl_name"].isin(clientes)]


In [14]:
clientes_h = df_clientes[["cl_name","date_invoice","product_name","quantity","price_subtotal"]]

In [15]:
clientes_h.to_excel("infromevisum.xlsx")